数据集：

In [5]:
dataSet = [[1, 1, 'yes'],
               [1, 1, 'yes'],
               [1, 0, 'no'],
               [0, 1, 'no'],
               [0, 1, 'no']]
labels = ['no surfacing','flippers']


In [15]:
import numpy as np

香农熵计算公式：
$$H=-\sum_{i=1}^np(x_i)log_2p(x_i)$$

In [36]:
def cal_shannon_ent(data):
    num_entries=len(data)
    label_counts={}
    ##获得每个label出现的个数
    for feat_vec in data:
        #print(feat_vec)
        current_label=feat_vec[-1]
        if current_label not in label_counts:
            label_counts[current_label]=1
        else:
            label_counts[current_label]+=1
    shannon_ent=0
   # print(label_counts)
    for key in label_counts:
        prob = label_counts[key]/num_entries
        shannon_ent-=prob*np.log2(prob)
    return shannon_ent

In [37]:
cal_shannon_ent(dataSet)

0.9709505944546686

In [26]:
def split_data(data,axis,value):
    ret_data=[]
    for feat_vect in data:
        if feat_vect[axis]==value:
            reduced_faeture=feat_vect[:axis]
            reduced_faeture.extend(feat_vect[axis+1:])
            ret_data.append(reduced_faeture)
    return ret_data

In [32]:
dataSet = [[1, 1, 'yes'],
               [1, 1, 'yes'],
               [1, 0, 'no'],
               [0, 1, 'no'],
               [0, 1, 'no']]
split_data(dataSet,0,0)

[[1, 'no'], [1, 'no']]

In [42]:
def choose_best_feature(data):
    num_features = len(data[0])-1
    base_ent = cal_shannon_ent(data)
    best_info_gain = 0
    best_feature = 1
    for i in range(num_features):
        
        feat_list  =  [x[i] for x in data]
        unique_value = set(feat_list)
        new_ent = 0
        # 计算每种划分方式的信息熵
        for value in unique_value:
            sub_data = split_data(data,i,value)
            prob = len(sub_data)/len(data)
            new_ent += prob*cal_shannon_ent(sub_data)
        info_gain = base_ent-new_ent
        # 选择最好的划分特征
        print(i,new_ent)
        if info_gain > best_info_gain:
            best_info_gain = info_gain
            best_feature = i
    return best_feature
            

In [43]:
choose_best_feature(dataSet)

0 0.5509775004326937
1 0.8


0

In [65]:
import operator
def majority_cnt(class_list):
    class_count={}
    for vote in class_list:
        if vote not in class_count:
            class_count[vote] = 1
        else:
            class_count[vote] += 1
            
    sorted_calss_cnt = sorted(class_count.keys(),key=operator.itemgetter(1),reverse=True)
    print('s',sorted_calss_cnt)
    return sorted_calss_cnt[0][0]

In [66]:
def create_tree(data,label):
    labels=label[:]
    class_list= [x[-1] for x in data]
    #当所有值一样时停止
    if class_list.count(class_list[0]) == len(class_list):
        return class_list[0]
    if len(data[0])==1:
        return majority_cnt(class_list)
    best_feat = choose_best_feature(data)
    best_feat_label =labels[best_feat]
    my_tree = {best_feat_label:{}}
    del labels[best_feat]
    feat_value = [x[best_feat] for x in data]
    unique_value = set(feat_value)
    for value in unique_value:
        sub_label = labels[:]
        my_tree[best_feat_label][value] = create_tree(split_data(data,best_feat,value),sub_label)
    return my_tree

In [67]:
create_tree(dataSet,labels)

0 0.5509775004326937
1 0.8
0 0.0


{'no surfacing': {0: 'no', 1: {'flippers': {0: 'no', 1: 'yes'}}}}

In [80]:
def classify(input_tree,feat_labels,test_vec):
    first_str=list(input_tree.keys())[0]
    second_dict= input_tree[first_str]
    feat_index = feat_labels.index(first_str)
    for key in second_dict.keys():
        if test_vec[feat_index] == key:
            if type(second_dict[key]).__name__ == 'dict':
                class_label = classify(second_dict[key],feat_labels,test_vec)
            else:
                class_label = second_dict[key]
    return class_label

In [70]:
my_tree = create_tree(dataSet,labels)

0 0.5509775004326937
1 0.8
0 0.0


In [79]:
list(my_tree.keys())

['no surfacing']

In [83]:
classify(create_tree(dataSet,labels),labels,[0,0])

0 0.5509775004326937
1 0.8
0 0.0


'no'

In [82]:
dataSet

[[1, 1, 'yes'], [1, 1, 'yes'], [1, 0, 'no'], [0, 1, 'no'], [0, 1, 'no']]